In [23]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [24]:
data_dir = 'imagens'
img_height = 300
img_width = 300
batch_size = 32

In [25]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size
)
class_names = train_ds.class_names

Found 2106 files belonging to 10 classes.
Using 1685 files for training.


In [26]:
train_ds = train_ds.map(lambda x,y:tf.image.rgb_to_grayscale(x))

In [27]:
val_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split = 0.2,
    subset = "validation",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

Found 2106 files belonging to 10 classes.
Using 421 files for validation.


In [28]:
val_ds = val_ds.map(lambda x,y:tf.image.rgb_to_grayscale(x))

In [29]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [30]:
num_classes = len(class_names)

model = Sequential([
  layers.Rescaling(1./255, input_shape=(img_height, img_width, 1)),
    layers.RandomFlip(),
    layers.RandomZoom(height_factor=0.3),
    layers.Conv2D(16, 1, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 1, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 1, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

In [31]:
model.compile(optimizer='adam',
             loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
             metrics=['accuracy'])

In [32]:
epochs = 100
history = model.fit(
    train_ds,
    validation_data = val_ds,
    epochs = epochs
)

Epoch 1/100


ValueError: in user code:

    File "C:\Users\yurid\carros_BR\venv\lib\site-packages\keras\engine\training.py", line 1051, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\yurid\carros_BR\venv\lib\site-packages\keras\engine\training.py", line 1040, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\yurid\carros_BR\venv\lib\site-packages\keras\engine\training.py", line 1030, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\yurid\carros_BR\venv\lib\site-packages\keras\engine\training.py", line 891, in train_step
        self._validate_target_and_loss(y, loss)
    File "C:\Users\yurid\carros_BR\venv\lib\site-packages\keras\engine\training.py", line 848, in _validate_target_and_loss
        raise ValueError(

    ValueError: Target data is missing. Your model was compiled with loss=<keras.losses.SparseCategoricalCrossentropy object at 0x0000016070083EB0>, and therefor